In [3]:
import cv2 as cv


import import_ipynb

import os
from utilities import*

<span style="color:#fff; font-family: 'Bebas Neue'; font-size: 3em;">Preprocessing Module</span>

In [4]:
def ThresholdImage(image,Trace=False):
    img=image.copy()
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # use THRESH_OTSU to detect the threshold value automatically
    img = cv.threshold(img, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)[1]
    num_ones = np.count_nonzero(img)
    num_zeros = img.size - num_ones

    # it means that background is white and text is black
    if num_ones > num_zeros:
        img = cv.bitwise_not(img)
    if Trace:
        print(num_ones, num_zeros)
        show_images([img], ["Thresholded Image White background and Black text"])
    return img

In [5]:
def RemoveSaltPepperNoise(image,Trace=False):
    img = image.copy()
    # Remove Salt and Pepper Noise
    img = cv.medianBlur(img, 3)
    if Trace:
        show_images([img], ["Image after removing Salt and Pepper Noise"])
    return img

In [20]:
def DeskewImage(image,real,Trace=False):

    original = image.copy()
    img = image.copy()

    kernel = cv.getStructuringElement(cv.MORPH_RECT, (3, 3))
    opening = cv.morphologyEx(img, cv.MORPH_OPEN, kernel)
    if Trace:
        show_images([opening], ["Opening Image"])
    # Dilate the text to make the text lines more obvious.
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (35, 35))
    dilate = cv.dilate(opening, kernel)
    if Trace:
        show_images([dilate], ["Dilated Image"])

    # Find all the contours based on the dilated image.
    contours, hierarchy = cv.findContours(
        dilate, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE
    )

    # show the contours
    if Trace:
        img = cv.drawContours(real, contours, -1, (0, 255, 0), 5)
        show_images([img], ["Contours"])

    # sort the contours based on the area
    contours = sorted(contours, key=cv.contourArea, reverse=False)
    # filter out the small contours
    contours = [cnt for cnt in contours if cv.contourArea(cnt)<0.9*img.shape[0]*img.shape[1]]  

    rect = cv.minAreaRect(contours[-1])
    angle = rect[-1]
    box = cv.boxPoints(rect)
    box = np.intp(box)

    if Trace:
        cv.drawContours(img,[box],0,(0,0,255),2)
        show_images([img], ["Rotated Rectangle"])

    # Calculate distances between consecutive points
    dis = []
    for i in range(len(box)):
        # Calculate the Euclidean distance between consecutive points
        dist = np.linalg.norm(box[i] - box[(i + 1) % len(box)])
        dis.append(dist)

    if Trace:
        print(f'Angle: {angle}')
        for (x,y) in box:
            print(f'({x},{y})')
        # for i, dist in enumerate(dis):
        #     print("Distance of side {}: {:.2f}".format(i + 1, dist))
    sortedx_box = sorted(box, key=lambda k: k[0])
    sortedy_box = sorted(box, key=lambda k: k[1])
    dis0 = np.linalg.norm(sortedy_box[-1] - sortedy_box[-2])
    dis1 = np.linalg.norm(sortedx_box[-1] - sortedx_box[-2])
    if Trace:
        print(f'sorted box')
        for (x,y) in box:
            print(f'sorted ({x},{y})')
        print(f"dis0: {dis0}")
        print(f"dis1: {dis1}")

    tolernace = 35

    if angle >= 80:
        if dis0 > dis1 +  tolernace:
            angle = 0
            return original
        elif  dis1 > dis0 +  tolernace:
            angle = 90
        else:    
            angle = 0
            return original
    elif angle >=35 and angle < 55:
        angle = 45
    elif angle <= 10:
        if dis0 > dis1 + tolernace:
            angle = 0
            return original
        elif dis1 > dis0 + tolernace:
            angle = 90
        else:
            angle = 0
            return original

    height = original.shape[0]
    width = original.shape[1]
    m = cv.getRotationMatrix2D((width / 2, height / 2), angle, 1)
    deskewed = cv.warpAffine(original, m, (width, height), borderValue=(0,0,0))

    return deskewed

In [21]:
def Preprocessing(image,Trace=False):
    img = image.copy()
    # Remove Salt and Pepper Noise
    img = RemoveSaltPepperNoise(img, Trace)
    # Threshold the image
    img = ThresholdImage(img, Trace)
    if np.count_nonzero(img) == 0:
        return None
    # Deskew the image
    deskewed = DeskewImage(img,image, Trace)
    return deskewed